Code based on Matt's implementation of Project #2, which includes starter code and utilities provided in the course

In [1]:
# Setup in VSCode
def setup():
    # %pip install -r ../requirements.txt
    %matplotlib widget
    
setup()

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from random import randrange

datadir = "resources/"

import utils
from utils import cut

In [4]:
# HELPER FUNCTIONS

def choose_sample(x, y, patch_size, sample, guidance_im, alpha, output_padded, mask_padded, tol=1, overlap=0, seam_cut = False, random = False, display_seam_cut_charts=False):
    seam_cut = False if random else seam_cut
    if random:
        patch_coords = get_patch_random(sample.shape[1], sample.shape[0], patch_size)
        patch = sample[patch_coords[0]:patch_coords[0] + patch_size, patch_coords[1]:patch_coords[1] + patch_size]
    else:
        T = output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size]
        M = mask_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size]
        ssd_image = ssd_image_generate(
            T=T, 
            M=M, 
            I=sample
        )
        ssd_guidance = ssd_image_generate(
            T=guidance_im[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size],
            M=M,
            I=sample
        )
        patch_coords = get_patch_ssd(
            ssd_image=ssd_image*(1-alpha) + ssd_guidance*alpha,
            tol=tol,
            patch_size=patch_size
        )
        patch = sample[patch_coords[0]:patch_coords[0] + patch_size, patch_coords[1]:patch_coords[1] + patch_size]
        if seam_cut:
            T_gray = cv2.cvtColor(np.array(T*255.0, dtype=np.uint8), cv2.COLOR_RGB2GRAY)
            is_on_left_column = x == 0
            is_on_top_row = y == 0
            left_seam = np.zeros(shape=(patch_size, patch_size), dtype=np.uint8)
            top_seam = np.zeros(shape=(patch_size, patch_size), dtype=np.uint8)
            if not is_on_left_column:
                left_seam[:,:overlap] = 1-utils.cut(T_gray[:,:overlap].T).T
            if not is_on_top_row:
                top_seam[:overlap,:] = 1-utils.cut(T_gray[:overlap,:])
            mask_combined_seams = np.logical_or(left_seam, top_seam).astype(np.uint8)
            if display_seam_cut_charts:
                display(f"{(is_on_left_column, is_on_top_row)}, {(y, patch_size)}")
                fig, ax = plt.subplots(1,6, figsize=(10,10))
                ax[0].imshow(T)
                ax[0].title.set_text("Two Overlapping\nPortions")
                ax[1].imshow(get_cost(T_gray))
                ax[1].title.set_text("Pixelwise\nSSD Cost")
                ax[2].imshow(top_seam)
                ax[2].title.set_text("Horizontal Mask")
                ax[3].imshow(left_seam)
                ax[3].title.set_text("Vertical Mask")
                ax[4].imshow(mask_combined_seams)
                ax[4].title.set_text("Combined Masks")
                mask_applied = T.copy()
                mask_applied[:,:,0] = T[:,:,0]*mask_combined_seams
                mask_applied[:,:,1] = T[:,:,1]*mask_combined_seams
                mask_applied[:,:,2] = T[:,:,2]*mask_combined_seams
                ax[5].imshow(mask_applied)
                ax[5].title.set_text("Mask Applied")
            M = mask_combined_seams
    
    if not seam_cut:
        output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size] = patch
    else:
        output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size,0] = output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size,0] * (mask_combined_seams) + patch[:,:,0]*(1-mask_combined_seams)
        output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size,1] = output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size,1] * (mask_combined_seams) + patch[:,:,1]*(1-mask_combined_seams)
        output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size,2] = output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size,2] * (mask_combined_seams) + patch[:,:,2]*(1-mask_combined_seams)
    mask_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size] = np.ones(shape=(patch_size, patch_size))
    return output_padded, mask_padded

def choose_sample_ALTERNATE(x, y, patch_size, sample, output_padded, mask_padded, tol=1, overlap=0, seam_cut = False, random = False, display_seam_cut_charts=False):
    seam_cut = False if random else seam_cut
    if random:
        patch_coords = get_patch_random(sample.shape[1], sample.shape[0], patch_size)
        patch = sample[patch_coords[0]:patch_coords[0] + patch_size, patch_coords[1]:patch_coords[1] + patch_size]
    else:
        T = output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size]
        M = mask_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size]
        ssd_image = ssd_image_generate(
            T=T, 
            M=M, 
            I=sample
        )
        patch_coords = get_patch_ssd(
            ssd_image=ssd_image,
            tol=tol,
            patch_size=patch_size
        )
        patch = sample[patch_coords[0]:patch_coords[0] + patch_size, patch_coords[1]:patch_coords[1] + patch_size]
        if seam_cut:
            is_on_left_column = x == 0
            is_on_top_row = y == 0
            T_seam = ssd_image_generate(
                    T = T,
                    M = np.ones(shape=(patch_size, patch_size), dtype=np.uint8),
                    I = patch
                )
            left_seam = np.zeros(shape=(patch_size, patch_size), dtype=np.uint8)
            top_seam = np.zeros(shape=(patch_size, patch_size), dtype=np.uint8)
            if not is_on_left_column:
                left_seam[:,:overlap] = 1-utils.cut(T_seam[:,:overlap].T).T
            if not is_on_top_row:
                top_seam[:overlap,:] = 1-utils.cut(T_seam[:overlap,:])
            mask_combined_seams = np.logical_or(left_seam, top_seam).astype(np.uint8)
            if display_seam_cut_charts:
                display(f"{(is_on_left_column, is_on_top_row)}, {(y, patch_size)}")
                fig, ax = plt.subplots(1,6, figsize=(10,10))
                ax[0].imshow(T)
                ax[0].title.set_text("Two Overlapping\nPortions")
                ax[1].imshow(T_seam)
                ax[1].title.set_text("Pixelwise\nSSD Cost")
                ax[2].imshow(top_seam)
                ax[2].title.set_text("Horizontal Mask")
                ax[3].imshow(left_seam)
                ax[3].title.set_text("Vertical Mask")
                ax[4].imshow(mask_combined_seams)
                ax[4].title.set_text("Combined Masks")
                mask_applied = T.copy()
                mask_applied[:,:,0] = T[:,:,0]*mask_combined_seams
                mask_applied[:,:,1] = T[:,:,1]*mask_combined_seams
                mask_applied[:,:,2] = T[:,:,2]*mask_combined_seams
                ax[5].imshow(mask_applied)
                ax[5].title.set_text("Mask Applied")
            M = mask_combined_seams
    
    if not seam_cut:
        output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size] = patch
    else:
        output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size,0] = output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size,0] * (mask_combined_seams) + patch[:,:,0]*(1-mask_combined_seams)
        output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size,1] = output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size,1] * (mask_combined_seams) + patch[:,:,1]*(1-mask_combined_seams)
        output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size,2] = output_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size,2] * (mask_combined_seams) + patch[:,:,2]*(1-mask_combined_seams)
    mask_padded[y+patch_size:y+2*patch_size, x+patch_size:x+2*patch_size] = np.ones(shape=(patch_size, patch_size))
    return output_padded, mask_padded

def ssd_image_generate(T, M, I):
    ssd_cost_r = ((M*T[:,:,0])**2).sum() - 2 * cv2.filter2D(I[:,:,0], ddepth=-1, kernel = M*T[:,:,0]) + cv2.filter2D(I[:,:,0] ** 2, ddepth=-1, kernel=M)
    ssd_cost_g = ((M*T[:,:,1])**2).sum() - 2 * cv2.filter2D(I[:,:,1], ddepth=-1, kernel = M*T[:,:,1]) + cv2.filter2D(I[:,:,1] ** 2, ddepth=-1, kernel=M)
    ssd_cost_b = ((M*T[:,:,2])**2).sum() - 2 * cv2.filter2D(I[:,:,2], ddepth=-1, kernel = M*T[:,:,2]) + cv2.filter2D(I[:,:,2] ** 2, ddepth=-1, kernel=M)
    ssd_map = np.array(ssd_cost_r + ssd_cost_g + ssd_cost_b)
    return ssd_map

def get_patch_ssd(ssd_image, tol, patch_size):
    patch_radius = int(patch_size*0.5)
    ssd_image = ssd_image.copy()[patch_radius:-patch_radius, patch_radius:-patch_radius]
    # find _tol_ max values in ssd_image
    max_coords = []
    for i in range(tol+1):
        # https://stackoverflow.com/questions/55284090/how-to-find-maximum-value-in-whole-2d-array-with-indices
        if len(ssd_image) == 0:
            display("SSD IMAGE == 0")
            plt.figure()
            plt.imshow(ssd_image)
        current_max_coords = np.unravel_index(np.argmin(ssd_image), (ssd_image.shape[0], ssd_image.shape[1]))
        ssd_image[current_max_coords] = 20
        max_coords.append(current_max_coords)
    out_coords = max_coords[randrange(0, tol)]
    return out_coords

def get_patch_random(width, height, patch_size):
    random_x = randrange(0, width - patch_size)
    random_y = randrange(0, height - patch_size)
    return (random_y, random_x)

In [5]:
from math import ceil
from ipywidgets import IntProgress

def texture_transfer(sample, patch_size, overlap, tol, guidance_im, alpha):
    seam_cut = True
    random = False
    debug = False
    tile_to_stop_at = -1
    # ensure odd patch_size
    if patch_size%2 != 1:
        patch_size += 1
        print(f"Patch size increased to {patch_size}!")
        
    if debug:
        tol=1
        display("DEBUG: TOL=1")
        
    # params
    num_tiles_x = ceil(guidance_im.shape[1]/(patch_size-overlap))
    num_tiles_y = ceil(guidance_im.shape[0]/(patch_size-overlap))
    
    sample, sample_w, sample_h = sample/255.0, sample.shape[1], sample.shape[0]
    output_padded = np.zeros(shape=(guidance_im.shape[0]+2*patch_size, guidance_im.shape[1]+2*patch_size,3), dtype=np.float32)
    mask_padded = np.zeros(shape=(guidance_im.shape[0]+2*patch_size, guidance_im.shape[1]+2*patch_size), dtype=np.uint8)
    guidance_padded = output_padded.copy()
    guidance_padded[patch_size:-patch_size,patch_size:-patch_size] = guidance_im/255.0
    
    # add random patch to the top left corner
    patch_x_random = randrange(0, sample_w - patch_size)
    patch_y_random = randrange(0, sample_h - patch_size)
    # output_padded[patch_size:2*patch_size, patch_size:2*patch_size] = sample[
    #     patch_y_random : patch_y_random + patch_size,
    #     patch_x_random : patch_x_random + patch_size
    # ]
    mask_padded[patch_size:2*patch_size, patch_size:2*patch_size] = np.ones(shape=(patch_size, patch_size))
    
    current_tile = 0
    
    progress_bar = IntProgress(min=0, max=num_tiles_y-1)
    display(progress_bar)
    
    for j in range(num_tiles_y):
        for i in range(num_tiles_x):
            # if i == 0 and j == 0:
            #     continue
            if tile_to_stop_at == -1 or tile_to_stop_at >= current_tile:
                # display((i*(patch_size-overlap),j*(patch_size-overlap)))
                output_padded, mask_padded = choose_sample(
                    x = i*(patch_size-overlap),
                    y = j*(patch_size-overlap),
                    patch_size=patch_size,
                    sample=sample,
                    guidance_im=guidance_padded,
                    alpha=alpha,
                    output_padded=output_padded,
                    mask_padded=mask_padded,
                    tol=tol,
                    seam_cut=seam_cut,
                    overlap=overlap,
                    random=random,
                    display_seam_cut_charts= tile_to_stop_at == current_tile
                )
            current_tile += 1
        progress_bar.value = j
    
    output = output_padded[patch_size:-patch_size, patch_size:-patch_size]
    output = (output*255.0).astype(np.uint8)
    # output = (output_padded*255.0).astype(np.uint8)
    return output

In [12]:
# load/process appropriate input texture and guidance images
texture_img_fn = datadir + '/samples/sketch.tiff'
texture_img = cv2.cvtColor(cv2.imread(texture_img_fn), cv2.COLOR_BGR2RGB)
guidance_img_fn = datadir + '/samples/feynman.tiff'
guidance_img = cv2.cvtColor(cv2.imread(guidance_img_fn), cv2.COLOR_BGR2RGB)
patch_size = 25
overlap = 11
tol = 3
alpha = 0.5
res = texture_transfer(texture_img, patch_size, overlap, tol, guidance_img, alpha)

cv2.imwrite("Test.png", res)

# plt.figure(figsize=(15,15))
# plt.imshow(res)
# plt.show()

IntProgress(value=0, max=25)

KeyboardInterrupt: 

In [30]:
style_src = "../resources/style/bazille_and_camille_(study_for__dejeuner_sur_l'herbe_)_1970.17.41.jpg"
style_img = cv2.cvtColor(cv2.imread(style_src), cv2.COLOR_BGR2RGB)
guidance_imgs_folder = "../resources/input-gan"

style_img = cv2.resize(style_img, (500, round(style_img.shape[0]/style_img.shape[1]*500)))
# plt.figure()
# plt.imshow(style_img)

from PIL import Image
import os

output_directory = "../resources/texture-transfer"

output_ext = "-tt.jpg"

N_inputs = len(os.listdir(guidance_imgs_folder))

for input_idx in range(N_inputs):
    input_src = guidance_imgs_folder + "/" + os.listdir(guidance_imgs_folder)[input_idx]
    # https://stackoverflow.com/questions/678236/how-do-i-get-the-filename-without-the-extension-from-a-path-in-python
    output_src = output_directory + "/"+ os.path.splitext(os.path.basename(os.listdir(guidance_imgs_folder)[input_idx]))[0] + output_ext
    input_img = cv2.cvtColor(cv2.imread(input_src), cv2.COLOR_BGR2RGB)
    input_img = cv2.resize(input_img, (500, round(input_img.shape[0]/input_img.shape[1]*500)))
    patch_size = 25
    overlap = 11
    tol = 3
    alpha = 0.5
    res = texture_transfer(style_img, patch_size, overlap, tol, input_img, alpha)
    cv2.imwrite(output_src, res)
    

IntProgress(value=0, max=47)

IntProgress(value=0, max=26)

IntProgress(value=0, max=35)

IntProgress(value=0, max=26)

IntProgress(value=0, max=47)

...

IntProgress(value=0, max=26)

IntProgress(value=0, max=47)

IntProgress(value=0, max=26)

IntProgress(value=0, max=34)

IntProgress(value=0, max=26)